## 2. Fasea: Algoritmoak diseinatzen

#### [Ikasle]

Community Detection proiektuaren 1. fasea entregatu duzue, eta feedback-a jaso ere. Klasean hainbat algoritmo ikusi ditugu, batzuk soluzio bakarrean oinarritutakoak, beste batzuk aldiz, populazio bat erabiltzen dutenak. Horiez gain, hibridatzeko teknikak ere ikusi ditugu. Bigarrengo fase honetan, hiru algoritmo diseinatu beharko dituzue. Lehenengoa, algoritmo eraikitzaile bat izango da. Bigarrena, soluzio bakarrean oinarritutako heuristiko bat izan beharko du, eta azkenik, hirugarrena algoritmo poblazional bat izango da. Hiru algoritmoak estokastikoak izan beharko dute, eta horietatik, bik, oinarri probabilistikoa izan beharko dute. Adibidez, Simulated Annealing, Estimation of Distribution Algorithms (EDAk) edota Ant Colony Optimization (ACO) implementatu ditzazkezue. Proiektu honen kasuan, algoritmoen helburua, komunitate kopuru jakin bat emanik, modularitatea maximizatzen duen komunitate banaketa (soluzioa) bilatzen saiatzea da.

Errepasatu gaitegian zehar ikusi ditugun algoritmo guztiak, eta horiek kontuak izanik, libre zarete nahi dituzuen diseinuak sortzeko, baita ere hibridoak! Adi! Egiten duzuen aukeraketa argudiatu egin beharko duzue.

#### Entregablea

Bigarrengo fasea ebaluatu ahal izateko, notebook honetan bertan algoritmoen diseinua eta implementazioa proposatu beharko duzue. Gogoratu algoritmo bat azaltzeko modurik errezena fluxu diagrama eta sasikode bat egitea direla. Adi! Atal bakoitzean hartutako erabakiak eta garatutako metodoak egoki argudiatu beharko dituzue. Azalpenak ere nahi ditut. Diagramak ez dira eurak bakarrik azaltzen, beraz testutik erreferentziatu egin beharko dituzue. Saiatu idazkera zientifiko-tekniko batekin idazten (pentsatu publikatuko duzuen lan bat dela). Ez argudiatzeak edo lana garaiz ez entregatzeak penalizazioa jasoko dute ebaluagarria den proiektuaren zati honetan. eGelan zehazten dira notebook-a igotzeko <b>egun eta orduak</b>.

Momentuz, ez daukazue algoritmoen exekuzio eta konparaketak egin behar. Hirugarren fasean, esperimentazioaren inguruko baldintzak emango dizkizuet, eta, horrez gain, txostenaren idazketa burutu beharko duzue.

In [5]:
import pandas as pd
from itertools import product
#from ipynb.fs.full.CDP_Sarrera_Ikasle import sortu_grafoa 
#import community
import networkx as nx
import numpy as np
import pandas as pd
import sqlite3
import networkx.algorithms.community as nx_comm
import itertools
from operator import itemgetter
from time import time
def sortu_grafoa():

    # Datuak irakurri
    # BETE HEMEN 8 lerro
    connect = sqlite3.connect(r"database.sqlite")
    query = """
    SELECT pa.paper_id, pa.author_id, a.name
    FROM paper_authors AS pa JOIN papers AS p ON pa.paper_id = p.id
    JOIN authors as a ON pa.author_id = a.id
    WHERE p.Year BETWEEN '2014' AND '2015'
    """
    df = pd.read_sql(query, connect)

    
    # Sortu grafoa
    # BETE HEMEN 7-10 lerro
    G = nx.Graph()

    for p, a in df.groupby('paper_id')['name']: 
        for u, v in itertools.combinations(a, 2):
            if G.has_edge(u, v):
                G[u][v]['weight'] +=1
            else:
                G.add_edge(u, v, weight=1)
    return G
def kalkulatu_A(G):
    time1 = time()
    
    m = len(G)
    authors = np.array(list(G.nodes()))

    A = nx.to_numpy_matrix(G)
    
    degrees = np.array(list(map(itemgetter(1), G.degree(authors)))) # Get array of degress according to authors

    min_degrees = np.min(degrees) #Calculate degrees matrix
    max_degrees = np.max(degrees) 
    
    degreesMatrix = np.zeros((max_degrees, np.shape(degrees)[0]))
    for i in range(max_degrees):
        degreesMatrix[i] = (i+1) * degrees /(2*m)
    
    authors_degreesMatrix = np.zeros((np.shape(authors)[0], np.shape(authors)[0]))
    
    for i in range(np.shape(authors)[0]):
        authors_degreesMatrix[i] = degreesMatrix[G.degree(authors[i]) -1]
    time2 = time()

    
    return  A - authors_degreesMatrix/(2*m)
def sigma(partizioa, G):
    m = len(G.nodes())
    
    partizio_kop = int(max(partizioa))
    a = map(lambda x: np.where(partizioa == x)[0], range(1, partizio_kop +1))
    
    partizioak = list(a)
    sigma_matrix = np.zeros((m, m))
    for i in partizioak[0]:
        np.put(sigma_matrix[i], partizioak[0], 1)
        
    return sigma_matrix
    
def modularitatea(G, partizioa,weight='weight'):
    ### Bete hemen 15-20 lerro
    m = len(G.nodes())
    return np.sum(np.multiply(sigma(partizioa, G), kalkulatu_A(G))) /(2 * m)

In [255]:
## Hemendik aurrera, intuizioa da nagusi...
import numpy as np
import random
#from ipynb.fs.full.CDP_Sarrera_Ikasle import sortu_grafoa 
#from ipynb.fs.full.CDP_1_fasea_Ikasle import kalkulatu_A, modularitatea
G = sortu_grafoa()
n = 2
v = 5
soluzioa_2= np.random.randint(low = 1, high = n +1, size =len(G))
soluzioa_4 = np.ones(int(len(G)))
soluzioa_3= np.array([1] * (int(len(G)/2) +1) + [2] * int(len(G)/2))
print(len(soluzioa_2))
soluzioa_3[0] = 2


def lehenengoakLortu(num_part):
    n = random.randint(num_part, num_part * 5)
    aukerak = list(range(1, num_part +1))
    l = np.array(sorted(G.degree, key = lambda x: x[1], reverse = True))
    l = l[0: n]
    a = np.array(list(map(int, l[:, 1])))
    
    suma = sum(a)
    prob = a / suma
    return l[np.random.choice(np.arange(0, n), p=prob, size =num_part, replace = False)]
    
#soluzioa_1 = np.ones(len(G))
#2 talde soilik sortuko ditugu
def get_key_from_value(d, val):
    keys = [k for k, v in d.items() if v in val]
    if keys:
        return keys[0]
    return 0

def eraikitzailea(G, num_part):
    soluzioa = np.zeros(len(G))
    authors = np.array(G.nodes())
    #HASIERAKETA
    
    lehenengoak = lehenengoakLortu(num_part)
    
    partizioak ={i: [] for i in range(num_part)}

    wait_authors_idx = list(range(len(authors)))
    for i in range(len(lehenengoak)):
        idx =np.squeeze(np.where(lehenengoak[i][0] == authors))
        soluzioa[idx] = i
        partizioak[i].append(idx)
        wait_authors_idx.remove(idx)
        
    #ERAIKITZAILEA
    random_nums = 0
    no = 0
    i = 0
    gehitua  = False
    while wait_authors_idx:
        k = wait_authors_idx[i]
        neighbor_idx = [np.squeeze(np.where(k == authors)) for k in G.neighbors(authors[k])]
        if len(neighbor_idx) == 1:
            no += 1

        vals = list(map(lambda x : get_key_from_value(partizioak, x), neighbor_idx))
        unique, counts = np.unique(vals, return_counts=True)
        unique = np.delete(unique, 0)
        counts = np.delete(counts, 0)

        vals= dict(zip(unique, counts))
        for j in vals:
            partizioak[j].append(k)
            wait_authors_idx.remove(k)
            i += -1
            gehitua = True
            break

        i += 1
        if i >= len(wait_authors_idx):
            i = 0
            print(len(wait_authors_idx))
            no  = 0
            if not gehitua:
                idx = random.randint(0, len(wait_authors_idx)-1)
                partizioak[random.randint(0, num_part-1)].append(wait_authors_idx[idx])
                del wait_authors_idx[idx]
                random_nums += 1

            gehitua = False            
        #print("i = ", i)
            
            #for j in neighbor_idx:
    print(random_nums)
    return partizioak
            
l = eraikitzailea(G, 3)

1843
1771
1661
1595
1516
1483
1482
1482
1481
1480
1475
1475
1471
1471
1467
1459
1458
1458
1457
1456
1455
1454
1446
1432
1414
1393
1391
1391
1388
1388
1387
1386
1380
1380
1375
1375
1373
1373
1361
1347
1325
1324
1324
1323
1322
1319
1319
1318
1317
1313
1307
1307
1306
1304
1304
1303
1300
1300
1296
1280
1274
1274
1269
1269
1267
1267
1266
1265
1264
1262
1262
1261
1260
1259
1258
1257
1252
1252
1251
1250
1248
1248
1247
1246
1240
1240
1236
1236
1235
1234
1228
1226
1226
1218
1217
1217
1212
1212
1211
1209
1209
1208
1207
1206
1199
1199
1198
1194
1194
1193
1187
1187
1186
1177
1177
1176
1167
1167
1166
1163
1162
1162
1160
1160
1159
1158
1155
1155
1151
1151
1144
1143
1143
1142
1141
1140
1133
1133
1126
1124
1123
1123
1122
1121
1118
1118
1117
1116
1107
1107
1104
1104
1103
1102
1099
1099
1098
1097
1096
1092
1092
1091
1090
1087
1086
1086
1085
1079
1079
1077
1077
1076
1075
1074
1070
1070
1069
1068
1065
1064
1060
1059
1059
1057
1057
1055
1055
1054
1053
1050
1050
1048
1048
1045
1045
1038
1038
1037
1036
1035


In [ ]:
def eraikitzailea(G,num_part):
    soluzioa = np.zeros(len(G))
    #Random first solution
    soluzioa[0] = np.random.randint(low =1, high =num_part+1)
    
#1.000.000 modularitate kalkulatzeko, 5.5h
#100.000 modularitate kalkulatzeko, 33min
#10.000 modularitate kalkulatzeko, 3min

In [22]:
import numpy as np
import copy

def insert(solution,G): #Ez dira tratatzen soluzio errepikatuak
    size = len(G)
    neigh = list()
    for i in range(size-1):
        for j in range(i+1,size):
            sol = solution.copy()
            solL = list(sol)
            ins = solL.pop(i)
            solL.insert(j,ins)
            sol_ar = np.array(solL)
            neigh.append(sol_ar)
    return neigh

def local_search_insert (G,sol, max_evals):
    size = len(G)
    best_solution = sol
    best_fitness = modularitatea(best_solution)
    evals = 1
    hobetu = True
    while hobetu and evals<max_evals:
        nei = insert(best_solution, G)
        last_f = modularitatea(G, best_solution)
        for i in nei:
            cur_fit = modularitatea(G, i)
            evals +=1
            if cur_fit<best_fitness:
                best_solution = i
                best_fitness = cur_fit
        if last_f == best_fitness:
            hobetu = False
    return (best_fitness, best_solution,evals)


def swapF(solution,i,j):
    sol = solution.copy()
    sol[i],sol[j] = sol[j],sol[i]
    return sol

def swap(solution,G): #Ez dira tratatzen soluzio errepikatuak
    size = len(G)
    neigh = list()
    for i in range(size):
        for j in range(i,size):
            another = swapF(solution,i,j)
            neigh.append(another)
    return neigh

def local_search_swap (G,sol, max_evals):
    size = len(G)
    best_solution = sol
    best_fitness = modularitatea(G, best_solution)
    evals = 1
    hobetu = True
    while hobetu and evals<max_evals:
        nei = swap(best_solution,G)
        last_f = modularitatea(G, best_solution)
        for i in nei:
            cur_fit = modularitatea(G, i)
            evals +=1
            if cur_fit<best_fitness:
                best_solution = i
                best_fitness = cur_fit
        if last_f == best_fitness:
            hobetu = False
            
    return (best_fitness, best_solution,evals)

In [23]:
def VND(G,initial_sol, max_evals):
    # BETE HEMEN 15-20 lerro
    best_solution = initial_sol
    best_fitness = modularitatea(G, best_solution)
    cur_evals=1
    hobetu = True
    while hobetu and cur_evals<max_evals:
        best_fitness,best_solution,evals = local_search_swap(G, best_solution, max_evals)
        cur_evals +=evals
        erdiko_fit = best_fitness
        best_fitness,best_solution,evals = local_search_insert(G,best_solution, max_evals)
        cur_evals +=evals
        if erdiko_fit==best_fitness:
            hobetu = False
    return (best_fitness, best_solution, cur_evals)

In [164]:
import random
def random_search(G, sol_kop):
    
    size = len(G)
    used_list = []
    best_fitness = 100000
    best_solution = []
    
    for i in range(sol_kop):
        badago = False
        sol = np.random.randint(0,num_part, size)
        for j in used_list:
            if j is sol:
                badago = True
                break
        if not badago:
            used_list.append(sol)
            value = modularitatea(G, sol)
            if value < best_fitness:
                best_fitness = value
                best_solution = sol
    return (best_fitness, best_solution)

In [133]:
VND(G, soluzioa_2, 20)

NameError: name 'VND' is not defined

In [211]:
modularitatea(G, np.array(a))

0.9077571714173919

In [247]:
from networkx.algorithms import community
authors = np.array(list(G.nodes()))
sol1 = community.kernighan_lin.kernighan_lin_bisection(G)
r = [set(authors[l[0]]), set(authors[l[1]]), set(authors[l[2]])]
print(modularitatea(G, np.array(a)))
nx_comm.modularity(G,r, weight="weight")

0.026576763608460025


0.2555219825034382